# Order Nakeds

In [1]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = "NSE"

# Set the root
from from_root import from_root
ROOT = from_root()

import pandas as pd
from loguru import logger

pd.options.display.max_columns = None
pd.set_option('display.precision', 2)

from pathlib import Path
import sys

# Add `src` and ROOT to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path

if "src" not in Path.cwd().parts:
    src_path = str(Path(get_path("purelib")) / "_src.pth")
    with open(src_path, "w") as f:
        f.write(str(ROOT / "src\n"))
        f.write(str(ROOT))
        if str(ROOT) not in sys.path:
            sys.path.insert(1, str(ROOT))

# Start the Jupyter loop
from ib_async import util

util.startLoop()

logger.add(sink=ROOT / "log" / "ztest.log", mode="w")

1

## Imports

In [2]:
from utils import handle_raws, get_pickle, load_config, arrange_orders, pickle_me, get_file_age, yes_or_no
from datetime import datetime
from ib_async import IB
from ibfuncs import get_open_orders, quick_pf, place_orders, make_ib_orders

## Set constants

In [3]:
config = load_config()
port = config.get('PORT')
MARGINPERORDER = config.get('MARGINPERORDER')

## Handle Raws

In [4]:
# Consolidate raw files to df_nakeds.pkl
handle_raws()

2024-08-05 07:43:53.829 | INFO     | utils:delete_files:155 - Deleted: C:\Users\kashi\python\nse\data\raw\earliest_nakeds1.pkl
2024-08-05 07:43:53.833 | INFO     | utils:remove_raw_nakeds:194 - Deleted files [WindowsPath('C:/Users/kashi/python/nse/data/raw/earliest_nakeds1.pkl')]


In [5]:
file_path = ROOT / 'data' / 'df_nakeds.pkl'

def how_many_days_old(file_path) -> float:
    """Gets the file's age in days"""
    file_age = get_file_age(file_path=file_path)
    
    seconds_in_a_day = 86400
    file_age_in_days = file_age.td.total_seconds() / seconds_in_a_day if file_age else 0
    
    return file_age_in_days

In [6]:
## Check the age of df_pickles, before ordering

In [7]:
txt = f"df_nakeds.pkl is {how_many_days_old(file_path): 0.2f}. Want to load?"
ans = yes_or_no(txt)

if ans:
    df_opts = get_pickle(file_path)
    print('\n\n')
    print(df_opts.drop(columns=['nse_symbol', 'instrument', 'contract', 'expiry']).head())
else:
    print('Bye!!!')




   ib_symbol  undPrice  safe_strike right  strike    dte    hv    iv   lot  \
0      PIIND   4255.00         4987     C  5000.0  24.43  0.31  0.37   250   
1  HINDUNILV   2696.00         2977     C  3000.0  24.44  0.23  0.22   300   
2  BAJAJFINS   1622.00         1804     C  1800.0  24.44  0.29  0.24   500   
3     CONCOR   1028.75         1205     C  1200.0  24.44  0.46  0.37  1000   
4  BALKRISIN   3293.45         3816     C  3800.0  24.44  0.36  0.34   300   

   price    sdev  intrinsic  bsPrice  comm   margin  xPrice    rom  
0  12.55  406.67        0.0     9.23  20.0  1767.44   12.55  26.52  
1   3.70  156.12        0.0     2.53  20.0  3264.98    3.70   5.08  
2   3.50  101.44        4.0     2.50  20.0  5366.02    7.50  10.44  
3   3.55   98.17        5.0     2.65  20.0  1580.74    8.55  80.79  
4  10.40  290.51       16.0     7.71  20.0  5373.20   26.40  22.01  


## check open orders

In [8]:
# check open orders
with IB().connect(port=port, clientId=10) as ib:
    dfo = get_open_orders(ib)
    dfp = quick_pf(ib)
    

In [ ]:
if not dfo.empty:
    remove_opens = set(dfo.symbol.to_list())
else:
    remove_opens = set()

In [10]:
# make a list of symbols to be removed from df_opts

if not dfp.empty:
    remove_positions = set(dfp.symbol.to_list())
else:
    remove_positions = set()

remove_ib_syms = remove_opens | remove_positions

# get the target options to plant
dft = df_opts[~df_opts.ib_symbol.isin(remove_ib_syms)].reset_index(drop=True)

In [11]:
len(dft)

1267

## Arrange and make orders

In [12]:
df_nakeds = arrange_orders(dft, maxmargin=MARGINPERORDER)
cos = make_ib_orders(df_nakeds)

## PLACE THE ORDER

# Archive the orders into `xn_history`

In [14]:
filename = f"{datetime.now().strftime('%Y%m%d_%I_%M_%p')}_naked_orders.pkl"
pickle_me(ordered, str(ROOT / "data" / "xn_history" / str(filename)))

In [ ]:
from ibfuncs import order_nakeds
order_nakeds(df_opts=df_opts, port=port)